# Análise de Sentimentos

Treinando a machine learning, utilizando o dataset recebido por email. (Faça o download e coloque a pasta dentro do repositorio)

Este notebook faz o treinando de machine learning de um modelo de análise de sentimentos, utilizando o dataset do Buscapé fornecido (confidencial), de forma que:
- Carrega e estrutura os dados do dataset em dicionário
- Exibe estatísticas acerca da do dataset
- Apresenta comentários acerca dos resultados obtidos

## Configurações Iniciais

Primeiramente, fazer o ajuste no path do Python para a raíz do projeto:

In [1]:
import os
os.chdir('..')

--------------------------------------

## Processamento do Dataset

Carregar a bliblioteca de normalização de texto e de dump de dados:

In [6]:
from nlputils.lexical.normalizer import Normalizer
import pickle

Instanciar um objeto para fazer as normalizações necessárias:

In [7]:
normalizer = Normalizer()

Implementar uma rotina de pré-processamento a ser aplicada em todo o dataset antes de construir o modelo e uma para auxiliar o carregamento dos dados, evitando replicação de código.

A função ```preprocessing``` recebe um texto (string) de entrada e o deixa normalizado, conforme procedimentos de nível lexical, e retorna o próprio texto (string) mais compacto.

In [8]:
def preprocessing(text):
    text = normalizer.to_lowercase(text)
    text = normalizer.remove_ponctuations(text)
    tokens = normalizer.tokenize_words(text)
    tokens = normalizer.remove_stopwords(tokens)
    return ' '.join(tokens)

A função ```all_files``` recebe o caminho do dataset a ser carregado e gera o caminho de todos os arquivos XML a serem lidos. O caminho de entrada deve terminal com ```/```.

In [9]:
def all_files(dataset_path):
    for directory in os.listdir(dataset_path):
        if directory != '.DS_Store':
            for stars in os.listdir(dataset_path + directory):
                for file in os.listdir(dataset_path + directory + '/' + stars):
                    if file.endswith('.xml'):
                        yield '{}{}/{}/{}'.format(dataset_path, directory, stars, file)

A função ```dumpvar``` salva em dados binário todo o valor de uma variável na memória. Basta fornecer o ponteiro do identificador da variável e o caminho onde será salvo o dump.

In [10]:
def dumpvar(var, path):
    path_itens = path.split('/')
    path_dir = '/'.join(path_itens[:-1]) + '/'
    
    if not os.path.isdir(path_dir):
        os.makedirs(path_dir)
    
    with open(path, 'wb') as fp:
        pickle.dump(var, fp)

A função ```loadvar``` retorna o valor de uma dump de memória salvo em disco. Basta o caminho onde está salvo o dump e atribuir em uma variável.

In [11]:
def loadvar(path):
    with open(path, 'rb') as fp:
        return pickle.load(fp)

---------------------------------

## Carregando o Dataset

Importar as bibliotecas utilizadas para fazer o parse do XML e criar a estrutura.

In [12]:
import xmltodict
from xml.parsers.expat import ExpatError
import pandas as pd
import time

Carregando os arquivos XML, exibindo estatísticas sobre eles e criando a estrutura que será utilizada no modelo de Machine Learning.

In [13]:
dataset = {'polarity': [], 'pros': [], 'cons': [], 'review': []}
count_ok = 0
count_err = 0

start_time = time.perf_counter()

for file in all_files('data/trainset/'):
    filename = file.split('/')[-1]
    with open(file, 'r') as text_file:
        data = text_file.read()
        try:
            dict_data = xmltodict.parse(data)
            polarity = float(dict_data['review']['stars']['@value'])
            dataset['polarity'].append(int(polarity))
            dataset['review'].append(str(dict_data['review']['opinion']))
            dataset['pros'].append(str(dict_data['review']['pros']))
            dataset['cons'].append(str(dict_data['review']['cons']))
            count_ok += 1
        except ExpatError:
            print('Arquivo "{}" está com xml mal formatado'.format(file))
            count_err += 1

end_time = time.perf_counter()

print('-------------------------------------------')
print('TEMPO DE EXECUÇÃO: {:.2f} segundos'.format(end_time - start_time))
print('Arquivos carregados OK:', count_ok)
print('Arquivos com ERRO:', count_err)

-------------------------------------------
TEMPO DE EXECUÇÃO: 20.04 segundos
Arquivos carregados OK: 67030
Arquivos com ERRO: 0


### Estatísticas acerca do dataset

Quantidade de comentários:

In [14]:
print(count_ok)

67030


Quantidade de sentenças:

In [15]:
pass

Quantidade de comentários por score:

In [16]:
for i in range(0, 6):
    print('Score #{}: {}'.format(i, dataset['polarity'].count(i)))

Score #0: 671
Score #1: 2396
Score #2: 2892
Score #3: 8937
Score #4: 25778
Score #5: 26356


---------------------------------

Aqui, o dataset está sendo repartido ao meio para criar o conjunto de treinamento e o de teste.

In [17]:
size_test = count_ok // 2
size_train = count_ok - size_test

dataset['trainset'] = ['train'] * size_train + ['test'] * size_test

In [18]:
dataframe = pd.DataFrame(data=dataset)
dataframe.head()

,cons,polarity,pros,review,trainset
0,Custo-Benefício\nDurabilidade\nQualidade do Ma...,3,Design,Se não fosse o uniforme do palmeiras seria uma...,train
1,Qualidade do Material\nConforto,3,Design\nCusto-Benefício,BBB\n\nO que gostei: GGG\n\nO que não gostei: ...,train
2,Durabilidade\nQualidade do Material,3,Design\nCusto-Benefício\nConforto,Um bom produto por um preço de melhor ainda.\n...,train
3,Design\nCusto-Benefício\nDurabilidade\nQualida...,3,Não possui nenhum ponto positivo.,Essa camiseta do time esfecifico é muito boa p...,train
4,Custo-Benefício,3,Não possui nenhum ponto positivo.,"NÃO TENHO MUiTO O QUE DiZER,AFiNAL COMPREi PAR...",train


Agora vamos ler todos os textos e adicionar um novo campo na dataframe que irá ter a sentença sem pontuações e stopwords

In [19]:
start_time = time.perf_counter()

dataframe['review'] = dataframe['review'].apply(preprocessing)
dataframe['pros'] = dataframe['pros'].apply(preprocessing)
dataframe['cons'] = dataframe['cons'].apply(preprocessing)

end_time = time.perf_counter()
print('TEMPO DE EXECUÇÃO: {:.2f} segundos'.format(end_time - start_time))

dataframe.head()

TEMPO DE EXECUÇÃO: 51.72 segundos


,cons,polarity,pros,review,trainset
0,custobenefício durabilidade qualidade material...,3,design,não uniforme palmeiras camisa bonita comprar c...,train
1,qualidade material conforto,3,design custobenefício,bbb gostei ggg não gostei gggg,train
2,durabilidade qualidade material,3,design custobenefício conforto,bom produto preço melhor ainda gostei bom prod...,train
3,design custobenefício durabilidade qualidade m...,3,não possui nenhum ponto positivo,camiseta time esfecifico é boa causa material ...,train
4,custobenefício,3,não possui nenhum ponto positivo,não dizerafinal comprei presente,train


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_reviews = dataframe[dataframe['trainset'] == 'train']['review'].values.tolist()
train_classes = dataframe[dataframe['trainset'] == 'train']['polarity'].values.tolist()
test_reviews = dataframe[dataframe['trainset'] == 'test']['review'].values.tolist()
test_classes = dataframe[dataframe['trainset'] == 'test']['polarity'].values.tolist()

transformer = TfidfVectorizer()
transformer.fit(train_reviews)
X = transformer.transform(train_reviews)
X_test = transformer.transform(test_reviews)

Treinando 

In [21]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [122]:
classifier = SVC()
classifier.fit(X, train_classes)

/home/bruno/Projetos/.virtualenvs/pln-ven/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [1]:
accuracy_score(test_classes, classifier.predict(X_test))

NameError: name 'accuracy_score' is not defined

In [22]:
classifier_lr = LogisticRegression()
classifier_lr.fit(X, train_classes)

/home/bruno/Projetos/.virtualenvs/pln-ven/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/bruno/Projetos/.virtualenvs/pln-ven/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [23]:
accuracy_score(test_classes, classifier_lr.predict(X_test))

0.4388184395046994

In [135]:
sentence = "Esse filme é fraco"
preprocessed_sentence = preprocessing(sentence)
print(preprocessed_sentence)
instance = transformer.transform([preprocessing(sentence)])
print(instance)
classifier_lr.predict(instance)

filme é fraco
  (0, 21978)	0.6504526192491843
  (0, 21348)	0.7595468320728326


array([3.])

In [136]:
sentence = "Esse filme é fraco"
preprocessed_sentence = preprocessing(sentence)
print(preprocessed_sentence)
instance = transformer.transform([preprocessing(sentence)])
print(instance)
classifier.predict(instance)

filme é fraco
  (0, 21978)	0.6504526192491843
  (0, 21348)	0.7595468320728326


array([5.])

In [3]:
from nlputils.semantics.sentiment import Sentiment

sentiment = Sentiment()
sentiment.sentiment_analysis('Esse filme é bom')

array([5])